### Terminology

* **Observation**: A sequence of images of the same field for a continuous amount of time. 
* **Frame**: A single image of a single field. An Observation consists of a sequence of frames.
* **Stamp**: A usually square stamp of pixels with a star near the center. One stamp corresponds to one time frame. A single frame contains many stamps.
* **PSC** Postage Stamp Cube: The collection of stamps corresponding to one particular source for the entire observation.

### Overview

The goal of the PIAA algorithm is to discover a change in brightness (flux) from a target star as compared to other stars in the same field that are similar to the target star.

### Steps

#### 1. Prepare Observation - Create PSC

This step prepares the images that are part of an observation. Images have already had the metadata ingested into the database.

* Plate-solve each image.
* Calibration:
    * Bias subtraction.
    * Flat fielding (_we are not taking flats yet_). Be very careful about how this done.
* Result: a PSC for each (valid) source in the field.
* Note: This step can happen before actual analysis for light-curves and should be done as part of data ingestion.

#### 2. Pick a target

The pipeline will eventually run the algorithm once for each PSC, doing a large number of comparisons. Here we "manually" select a target that we are interested in analyzing, particularly one that corresponds with an a known transit.

#### 3. Find "similar" stars

We want to find stars that are similar to our target in terms of how they are placed on the detector (e.g. does the star fall mostly on a red pixel or a blue, etc.). 

For this we ignore the flux and directly compare the morphology of each source with the target.

* Normalize each stamp of every PSC
* Find the sum-squared-difference for each PSC when compared to the target PSC.
* Result: A score for each point source that indicates how closely it matches the shape of the target source.

#### 4. Combine related stars

Taking the sources that are most morphological similar to our target, we linearly combine the 

We take the top matches* of the normalized sources and linearly combine them to match the target morphology.

The result of this step is a set of coefficients, one per matching source, that can be used to create the ideal template.

_* How we determine the "top" matches is an open question. For now we just take the 50-80 best matches_

#### 5. Create ideal template

The coefficients from step 4 are then applied to the original source stamps (rather than the normalized versions). These are linearly combined to form the ideal template.

The result of this step is one "ideal" template PSC that has the same shape as the target PSC.

#### 6. Compare target to template

The template PSC is then divided out of the target PSC, meaning there is a pixel-to-pixel division. If the template exactly matches the target then this will result in a stamp of all ones.

The real result will be a ratio of the target to the template, which by itself doesn't tell us much. However, as that ratio changes from frame _i_ to _i+1_ there will be a relative change between the target and the template.